In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.1 MB/s eta 0:00:00


In [7]:
!pip install PyPDF2 #helps read from pdf files
!pip install faiss-cpu
!pip install tiktoken #creates token

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [8]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [33]:
pdfreader = PdfReader('Writing Sample - Paper.pdf')

In [34]:
#reading text from pdf

from typing_extensions import Concatenate

raw_content = ""

for i , page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_content += content

In [35]:
raw_content

'De-SaTE: Denoising Self-attention Transformer\nEncoders for Li-ion Battery Health Prognostics\nGaurav Shinde∗, Rohan Mohapatra†, Pooja Krishan†, and Saptarshi Sengupta†\n∗Department of Engineering, San Jos ´e State University, San Jose, CA, USA\n†Department of Computer Science, San Jos ´e State University, San Jose, CA, USA\nEmail:∗gauravyeshwant.shinde@sjsu.edu,†rohan.mohapatra@sjsu.edu,\n†pooja.krishan@sjsu.edu,†saptarshi.sengupta@sjsu.edu\nAbstract —The usage of Lithium-ion (Li-ion) batteries has\ngained widespread popularity across various industries, from\npowering portable electronic devices to propelling electric vehi-\ncles and supporting energy storage systems. A central challenge\nin Li-ion battery reliability lies in accurately predicting their\nRemaining Useful Life (RUL), which is a critical measure\nfor proactive maintenance and predictive analytics. This study\npresents a novel approach that harnesses the power of multiple\ndenoising modules, each trained to address spe

In [36]:
text_splitter = CharacterTextSplitter(
    separator= '\n',
    chunk_size= 1300 ,
    chunk_overlap=200, #import for context
    length_function=len

)

In [37]:
texts = text_splitter.split_text(raw_content)

In [38]:
len(texts)

36

In [39]:
embeddings = OpenAIEmbeddings()

In [40]:
document_search = FAISS.from_texts(texts,embeddings)

In [41]:
document_search

In [42]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [43]:
chain = load_qa_chain(OpenAI(),chain_type="stuff")

In [45]:

query = "What is the key standpout point in the De-SaTE paper"
docs = document_search.similarity_search(query)
chain.run(input_documents = docs, question = query )

' The key standout point in the De-SaTE paper is the use of multiple denoising modules, each trained to address specific types of noise commonly encountered in battery data, to enhance data representation and accuracy in predicting Remaining Useful Life (RUL) of Lithium-ion (Li-ion) batteries.'